<a href="https://colab.research.google.com/github/ayarelif/DS-Unit-2-Linear-Models/blob/master/Elif_Ayar__of_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [ ]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [ ]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [ ]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [ ]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [ ]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


Before moving in depth, Let's clean data.

We are not going to use every columns to make a prediction. Thus the columns we could us are "features= ['Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap']". Thus Let's check only this columns

Before splitting the dataset, let's check the missing values at the target 'Great'

In [ ]:
df[['Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap']].isnull().sum()

Tortilla         0
Temp            20
Meat            14
Fillings         3
Meat:filling     9
Uniformity       2
Salsa           25
Synergy          2
Wrap             3
dtype: int64

In [ ]:
# Let's clean the column Temp first
df['Temp'].dropna(inplace=True)

In [ ]:
df['Temp'].isnull().sum()

0

In [ ]:
df['Meat'].dropna(inplace=True)

In [ ]:
df['Meat'].isnull().sum()

0

In [ ]:
df[['Temp', 'Meat']].isnull().sum()

Temp    20
Meat    14
dtype: int64

In [ ]:
df['Great'].isnull().sum()

0

In [ ]:
df['Great'].value_counts(dropna=False)

False    239
True     182
Name: Great, dtype: int64

##Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.


In [ ]:
from sklearn.model_selection import train_test_split


We need to check the date type and separate the years

In [ ]:
# we will choose the train set the years before 2016 and 2016
df['Date'].dtype


dtype('O')

In [ ]:
#convert the data type to time series

df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True)
df['Year']=df['Date'].dt.year
df[['Year', 'Date']].head()

,Year,Date
0,2016,2016-01-18
1,2016,2016-01-24
2,2016,2016-01-24
3,2016,2016-01-24
4,2016,2016-01-27


In [ ]:
train=df[df['Year']< 2017]
train[['Year', 'Date']].head()

,Year,Date
0,2016,2016-01-18
1,2016,2016-01-24
2,2016,2016-01-24
3,2016,2016-01-24
4,2016,2016-01-27


In [ ]:
validate=df[df['Year']==2017]
validate[['Year', 'Date']].head()

,Year,Date
301,2017,2017-01-04
302,2017,2017-01-04
303,2017,2017-01-07
304,2017,2017-01-07
305,2017,2017-01-10


In [ ]:
test=df[df['Year']>2017]
test[['Year', 'Date']].head()

,Year,Date
77,2026,2026-04-25
386,2018,2018-01-02
387,2018,2018-01-09
388,2018,2018-01-12
389,2018,2018-01-12


Thus All my data sets are ready: Train, test, and validate

##Begin with baselines for classification.

In [ ]:
# Determine the target and set the most frequent value

target='Great'
y_train=train[target]
y_train.value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

In [ ]:
#Same for Validate set
y_val=validate[target]
y_val.value_counts(normalize=True)

False    0.552941
True     0.447059
Name: Great, dtype: float64

False has the most majority of response with 59%. Thus, our base line is 0.59

In [ ]:
majority=y_train.mode()[0]
y_pred=[majority]*len(y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_train,y_pred)

0.5906040268456376

## Use scikit-learn for logistic regression.

In [ ]:
# 1) Imput estimator class

import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 2) Instantiate this class

log_reg=LogisticRegressionCV()

In [ ]:
# 3) Arrange X feature matrices (already did y target vectors)

train.columns

Index(['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
       'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
       'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
       'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
       'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
       'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Queso',
       'Egg', 'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini',
       'Great', 'Year'],
      dtype='object')

In [ ]:
features= ['Tortilla',
       'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa',
       'Synergy', 'Wrap']
X_train = train[features]
y_train = train[target]
X_val = validate[features]
y_val = validate[target]


In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((298, 9), (298,), (85, 9), (85,))

In [ ]:
# Impute missing values

from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')

X_train_imputed=imputer.fit_transform(X_train)
X_val_imputed=imputer.transform(X_val)


In [ ]:
# Let's use Scale function too
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

In [ ]:
# 4. Fit the model

log_reg.fit(X_train_scaled,y_train)

print('Validation Accuracy', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 0.4470588235294118


In [ ]:
# And prediction
log_reg.predict(X_val_imputed)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [ ]:
# Let's check coefficiency and intercept

log_reg.coef_

array([[0.31599287, 0.27882812, 0.59090665, 0.60834664, 0.48512659,
        0.1466913 , 0.2733873 , 0.68870353, 0.07858033]])

In [ ]:
log_reg.intercept_

array([-0.73738738])

I am not going to run and try with different features because accuracy value is pretty confident to use this model

##Get your model's test accuracy. (One time, at the end.)

In [ ]:
# Arrange the X features for test set
X_test=test[features]
y_test=test[target]

In [ ]:
# impute the missing values
X_test_imputed=imputer.transform(X_test)
X_test_scaled=scaler.transform(X_test_imputed)

In [ ]:
# Fit the model and find the validation score for test set

log_reg.fit(X_train_imputed,y_train)
print('Validation Score', log_reg.score(X_test_scaled,y_test))

Validation Score 0.42105263157894735
